# Notebook for PSTAT 235 Project


## Predicting Property Hype   
## Authors: 
    - Jonas Lundgren
    - Georgia Titcomb
    - Drew Pritchard
    - Xining Li

Contents
1. Data Cleaning
        1.1 Check for errors in columns
        1.2 Impute extreme values
2. Exploratory Data Analysis
        2.1 Bathrooms and bedrooms
        2.2 Price
        2.3 Latitude and Longitude
        2.4 Time created
        2.5 Features (amenities)
        2.6 Manager
        2.7 Description
3. Feature Engineering
        3.1 Features (amenities)
        3.2 Manager ID
        3.3 Time created
        3.4 Latitude and Longitude
        3.5 Description
                3.5.1 Using Count Vectorizer
                3.5.2 Using TF-IDF
                3.5.3 Using Word2Vec
4. Model creation and parameter tuning
        4.1 Baseline model
                4.1.1 Logistic
                4.1.2 Random Forest
                4.1.3 Decision Tree
        4.2 Amenities
                4.2.1 Logistic
                4.2.2 Random Forest
                4.2.3 Decision Tree
        4.3 Description
                4.3.1 Logistic
                4.3.2 Random Forest
                4.3.3 Decision Tree
        4.4. Mega model
                4.4.1 Logistic
                4.4.2 Random Forest
                4.4.3 Decision Tree
5. Evaluation
6. Additional exploratory work
        6.1 Time
        6.2 Neighborhoods
        6.3 Symbols
        6.4 Photos

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.linalg import DenseVector
import pyspark.sql.functions as F
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.types as typ
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql.functions import collect_set
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Bucketizer
from FeaturesMakers.OutlierSmoother import OutlierSmoother #Xining's smoother



import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Jonas_rentalPrice-Copy1") \
    .config("spark.executor.memory", '4g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [2]:
# import as pandas df
data_pd = pd.read_json("data/train.json")

In [3]:
# look at variables
data_pd.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,low


# 1 Data Cleaning

## 1.1 Check Columns for Errors


In [98]:
## Summary statistics for numerical columns
data_pd.describe()

,bathrooms,bedrooms,latitude,listing_id,longitude,price
count,49352.00000,49352.000000,49352.000000,4.935200e+04,49352.000000,4.935200e+04
mean,1.21218,1.541640,40.741545,7.024055e+06,-73.955716,3.830174e+03
std,0.50142,1.115018,0.638535,1.262746e+05,1.177912,2.206687e+04
min,0.00000,0.000000,0.000000,6.811957e+06,-118.271000,4.300000e+01
25%,1.00000,1.000000,40.728300,6.915888e+06,-73.991700,2.500000e+03
50%,1.00000,1.000000,40.751800,7.021070e+06,-73.977900,3.150000e+03
75%,1.00000,2.000000,40.774300,7.128733e+06,-73.954800,4.100000e+03
max,10.00000,8.000000,44.883500,7.753784e+06,0.000000,4.490000e+06


The average house has 1 bedroom and 1.5 bathrooms and is priced at about \$3800. Additionally, looking at the min and max values for these columns, we can see that there are some rentals that have 0 bedrooms or 0 bathrooms! The minimum rental price is 43 dollars, and the maximum is 4.5 million. Additionally, we can see some outliers in the latitude and longitude values. There is at least one place listed as being on the equator, and another that has latitude close to that of Vermont. Furthermore, there is at least one place with longitude = 0 (the prime meridian that runs through the UK and West Africa), and one with a longitude of -118.27, which is approximately the longitude of Los Angeles! There's definitely some cleaning to do here!

In [99]:
# Does every listing have a date?
import time
import datetime
set([0<time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()) for s in data_pd["created"]])

{True}

And all houses in NYC have a posted date

In [100]:
# descriptions
set([len(t.strip())>0 for t in data_pd["description"].tolist()])

{False, True}

In [101]:
# addresses
set([len(t.strip())>0 for t in data_pd["display_address"].tolist()])

{False, True}

In [102]:
# features
set([len(t)>0 for t in data_pd["features"].tolist()])

{False, True}

Some rentals have empty descriptions, addresses, or features

In [103]:
set([t>0 for t in data_pd["latitude"].tolist()])

{False, True}

We know NYC is about 40N, 74W, so rentals with latitude equal to 0 are likely to be errors. We will impute these values.

In [104]:
set([t<0 for t in data_pd["longitude"].tolist()])

{False, True}

In [105]:
set([t>0 for t in data_pd["price"].tolist()])

{True}

All houses are not free, which is good

In [106]:
set([len(t)>0 for t in data_pd["street_address"].tolist()])

{False, True}

There are some observations that do not have **street_address**

In [107]:
set([np.isreal(t) for t in data_pd["listing_id"].tolist()])

{True}

In [108]:
len(list(set(data_pd["listing_id"].tolist())))==len(data_pd["listing_id"].tolist())

True

Each **listing_id** is unique, which is expected

In [109]:
set([t for t in data_pd["interest_level"].tolist()])

{'high', 'low', 'medium'}

Each observation must have an **interest_level** in (high, low, medium)

## 1.2 Impute Values for Extreme Long/Lat

In [3]:
train_df = spark.createDataFrame(data_pd)

cols = [c for c in train_df.columns if (c == 'bathrooms' or c == 'bedrooms' or c == 'latitude' or c == 'longitude' or c == 'price')]   # exclude id from features
bounds = {} # will store lower and upper bounds for each feature

#Get bounds for each column
for col in cols:
    quantiles = train_df.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]

    bounds[col] = [
     quantiles[0] - 1.5 * IQR,
     quantiles[1] + 1.5 * IQR
    ]

#Label outliers
outliers = train_df.select(*['building_id'] + [
 (
 (train_df[c] < bounds[c][0]) | (train_df[c] > bounds[c][1]))
    .alias(c + '_outlier') for c in ['latitude', 'longitude']
])

#Create new columns of labels
c = 'latitude'
test = train_df.withColumn('latitude_outlier', ((train_df[c] < bounds[c][0]) | (train_df[c] > bounds[c][1])))


c = 'longitude'
test = test.withColumn('longitude_outlier', ((train_df[c] < bounds[c][0]) | (train_df[c] > bounds[c][1])))


In [ ]:
from pyspark.sql.functions import stddev as _stddev, col
from pyspark.sql.functions import abs

#Get medians of each column
med_lat = train_df.approxQuantile('latitude', [.5], .05)[0]
med_long = train_df.approxQuantile('longitude', [.5], .05)[0]

#If latitude_outlier[i] is True, then set to median, otherwise keep it
#Do same for longitude
test = test.withColumn('latitude', \
                          F.when(test.latitude_outlier == True, med_lat) \
                       .otherwise(test['latitude']))
                       
test = test.withColumn('longitude', \
               F.when(test.longitude_outlier == True, med_long) \
               .otherwise(test.longitude))

train_df = test.drop('latitude_outlier', 'longitude_outlier')

data_pd = train_df.toPandas()

## 1.3 Empty Strings in Description

In [ ]:
# How many blanks?
data_pd["word_count"] = data_pd['description'].str.split().str.len()
data_sparkdf = spark.createDataFrame(data_pd)

In [ ]:
49352 - data_pd["word_count"].astype(bool).sum(axis=0)

There are 3333 rentals with no words in their listings. However, each empty listing is not simply a "nan", '', or " "; several include multiple empty spaces. Therefore, we created a function to deal with these:

In [ ]:
from pyspark.sql.functions import when, lit, col

def replace(column1, column2, value):
    return when(column1 != value, column2).otherwise(lit("none"))

data_sparkdf2 = data_sparkdf.withColumn("description", replace(col("word_count"),col("description"), 0))

In [ ]:
data_pd = data_sparkdf2.toPandas() # convert back to pandas df

# 2 Exploratory Data Analysis

We will use a pandas dataframe for plotting, since it offers much for functionality for visualizations than pyspark dfs.

In [ ]:
interest_level_count = dict()
for j in [t for t in data_pd["interest_level"].tolist()]:
    interest_level_count[j] = interest_level_count.get(j,0) + 1
interest_level_count

We have counted the number of medium, low, and high. And as you can see, The data is a little bit imbalanced but not severe. 

## 2.1 bathrooms and bedrooms

We start by plotting some hisograms to see if anything stands out.

In [ ]:
data_pd["bathrooms"].hist()

In [ ]:
data_pd["bedrooms"].hist()

Continue with plotting.

In [ ]:
from matplotlib import pyplot as plt
plt.plot(data_pd["bedrooms"], data_pd["bathrooms"], 'o')

From this plot, we can see there is a strange house that has 10 bathrooms but only 2 bedrooms, which warrants a closer look.

In [ ]:
data_pd[data_pd["bathrooms"]==10]["description"].tolist()[0]

We check the images assosiated with the rental unit.

In [ ]:
from PIL import Image
import requests
from io import BytesIO

fig = plt.figure()
img_urls = data_pd[data_pd["bathrooms"]==10]["photos"].tolist()

for i in range(0, len(img_urls[0])):
    response = requests.get(img_urls[0][i])
    img = Image.open(BytesIO(response.content))
    ax = fig.add_subplot(3, 2, i+1)
    ax.imshow(img)
    
plt.show()

After checking thses images, we conclude that 10 bathrooms is an entry mistake. Continue to check the correlation between the two.

In [ ]:
from sklearn.linear_model import LinearRegression
import scipy
scipy.stats.pearsonr(data_pd["bathrooms"], data_pd["bedrooms"])

The **bathrooms** and **bedrooms** are correlated, as expected.

Most NYC rentals have fewer than three bedrooms and no more than two bathrooms.

In [ ]:
import seaborn
seaborn.catplot(x="interest_level", y="bathrooms", hue = "bedrooms",data = data_pd, order=["low","medium","high"])

In [ ]:
seaborn.catplot(x="interest_level", y="bedrooms", hue = "bathrooms",data = data_pd, order=["low","medium","high"])

We have observed that if there are more than 5 bathrooms or more than 4 bathrooms in a house, the **interest_level** associated with that house is rarely high.

## 2.2 price

We now investigate the price of for the rental units.

In [ ]:
mypal = {"low": "#6da39c", "medium": "#ff9a16", "high":"#77043e"}

We plot the `interest_level` vs the `price`.

In [ ]:
g = seaborn.catplot(x="interest_level", y="price",data = data_pd, order=["low","medium","high"], palette=mypal).set(ylim=(0, 50000))

And we can see the housing rental prices in NYC is very expensive. However, if a house's rental price is more than 10000, the interest level can rarely be high. 

## 2.3 latitude and longitude

First let's plot the longitude and latitude of the units and color based on the interest level.

In [ ]:
plt.subplots(figsize = (8, 8))
sns.scatterplot(x="longitude",
                y="latitude",
                hue = "interest_level",
                data = data_pd)

We can see an unnatural cross form in our data. This is from imputing either latitude or longitude depending on whether each was flagged as an outlier or not. Although it gives some innaccuracies in our plot, for modeling we wouldn't want all outliers to have the same lat/long coords in the center of our data.

Apart from that, it's hard to see if there are any spatial groupings in the data based on interest level.

Let's split the data into three plots for each interest_level class.

In [ ]:
plt.subplots(figsize = (12, 12))
sns.scatterplot(x = 'longitude',
                y = 'latitude',
                data = data_pd[data_pd.interest_level == 'low'],
                color='blue',
                alpha=0.1)
plt.subplots(figsize = (12, 12))
sns.scatterplot(x = 'longitude',
                y = 'latitude',
                data = data_pd[data_pd.interest_level == 'medium'],
                color='orange',
                alpha=0.1)
plt.subplots(figsize = (12, 12))
sns.scatterplot(x = 'longitude',
                y = 'latitude',
                data = data_pd[data_pd.interest_level == 'high'],
                color='red',
                alpha=0.1)

When we split them up we get a visual representation of the class imbalance between the interest levels. However, it's still unclear if these classes are forming groups dinstinct from the others. 

We want to use a clustering algorithm on the coordinates to see if there is predictive power within that data. Therefore, a gaussian mixture model or k-means may help in this case. 

To plot the apartments we use the package `folium` which can make interactive plots and we can get more information of the rental units in the plot.

Inspiration from blogpost: https://nbviewer.jupyter.org/github/vincentropy/python_cartography_tutorial/blob/master/part1_basic_folium_maps.ipynb

Uncomment to install `folium`.

In [ ]:
!pip install --upgrade folium
import folium

We create a function that places each of the units on the map. The color of the apartment corresponds to the factor `interest_level` which has not been investigated thoroughly yet. The size of the circle is based on the price which can be displayed by *klicking* the circle, along with the `street_address`.

In [ ]:
def plot_rental_locatinos(rental_locatinos):
        # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='50%')

            
    for index, row in rental_locatinos.iterrows():
        
            popup_text = "{}<br> Price: {}<br>"
            popup_text = popup_text.format(row["street_address"], row["price"])
            
            if row["interest_level"] == 'high':
                color="#77043e" # high
            if row["interest_level"] == 'medium':
                color="#ff9a16" # medium 
            if row["interest_level"] == 'low':
                color="#6da39c" # low
                
            radius = row["price"]/1000
        
            folium.CircleMarker(location=(row["latitude"],
                                          row["longitude"]),
                                          radius=radius,
                                          color=color,
                                          popup=popup_text,
                                          fill=True).add_to(folium_map)
    return folium_map

Since our data set is too big for all rental units to be displayed a subsample of 1000 units are changed into a pandas data frame which is then plotted using the above constructed function `plot_rental_locatinos`.

In [ ]:
sample_pd = data_pd.sample(n = 1000)

The user constructed function `plot_rental_locatinos` is used on the `sample_df`. Try draging, scrolling and moving around and klick different rental units for some basic information.

In [ ]:
plot_rental_locatinos(sample_pd)

## 2.4 Time Created

We now investigate when listing was created.

In [ ]:
## * NOTE: I am getting an error here

In [ ]:
createdUnixTime = list(map(lambda s:time.mktime(datetime.datetime.strptime(s, 
                                                         "%Y-%m-%d %H:%M:%S").timetuple()), 
         data_pd["created"].tolist()))

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=createdUnixTime, 
                            bins=20, 
                            color='#0504aa',
                            alpha=0.7, rwidth=2)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

In [ ]:
data_pd["createdUnixTime"] = createdUnixTime

In [ ]:
seaborn.catplot(x="interest_level", y="createdUnixTime",data = data_pd)

As you can see the created time for **high** has some gaps that **medium** and **low** does not have, so we may be able to utilize that after some careful feature engineering. 

## 2.5 features

The column called `features` (the naming has caused some confusion to say the least!) consists of different features assosiated with the rental unit. For example: _doorman, cats allowed, loft_.

We plot the number of features for the different units.

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=[len(t) for t in data_pd["features"].tolist()], 
                            bins=20, 
                            color='#0504aa',
                            alpha=0.7, rwidth=2)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Number of features')
plt.ylabel('Frequency')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

And most rows in the **features** column have less than 10 items

In [ ]:
data_pd["features_len"] = list(map(len, data_pd["features"].tolist()))

Here are the numbers of features assosiated with the different interest levels.

In [ ]:
seaborn.catplot(x="interest_level",
                    y="features_len",
                    data = data_pd)

And seems like the length of the **features** does not help...

# 2.6 Description

The column named 'description' contains a string with a narrative about the rental property. First we will simply look at the length of the descriptions and then we will generate word clouds to compare the relative importance of certain words in the description.

In [ ]:
data_pd.hist("word_count","interest_level", bins=50) 

For the most part, the distributions appear similar - most listings have word counts around 80-100 words, and this does not vary by interest level. However, one thing that does jump out immediately is that the proportion of listings with no description (word count=0) is much higher for low interest properties than for medium and high interest properties. 

To make word clouds, we first removed common words that didn't add much to our understanding of what was going on. After viewing these, we also added 'bedroom' and 'apartment' since these words also dominated the word clouds.

In [ ]:
add_stopwords = ["a","the","it","of","the","is","and","A","this","in","for", "with", "bedroom", "apartment"] 

In [ ]:
!pip install wordcloud # <----- uncomment to install
from wordcloud import WordCloud

low_data = data_pd[data_pd["interest_level"]== "low"]
low_data.head(2)

lowwordcloud = WordCloud(
                          background_color='white',
                          stopwords=add_stopwords, # common in all descriptions
                          max_words=200,
                          #max_font_size=40, 
                          random_state=42
                         ).generate(str(low_data['description']))

plt.imshow(lowwordcloud)
plt.axis('off')
plt.show()
med_data = data_pd[data_pd["interest_level"]== "medium"]
medwordcloud = WordCloud(
                          background_color='lightgray',
                          stopwords=add_stopwords,
                          max_words=200,
                          #max_font_size=40, 
                          random_state=42
                         ).generate(str(med_data['description']))
plt.imshow(medwordcloud)
plt.axis('off')
plt.show()
high_data = data_pd[data_pd["interest_level"]== "high"]
highwordcloud = WordCloud(
                          background_color='darkgray',
                          stopwords=add_stopwords,
                          max_words=200,
                          #max_font_size=40, 
                          random_state=42
                         ).generate(str(high_data['description']))
plt.imshow(highwordcloud)
plt.axis('off')
plt.show()

Above, word clouds are show from low to high interest levels. It looks like rentals that are described as one bedroom studios are generating the most interest. Additionally, we can see that upper east and west are showing up in the word clouds of medium and high interest properties, indicating that these neighborhoods might also be important. Additionally, 'no fee' shows up in medium and high interest word clouds, indicating that a bigram analysis might be helpful later on.

# 3 Feature Engineering

The different columns in the data set that will be used for feature engineering are:
- `features`
- `manager_id`
- `latitude and longitude`
- `description`
- `images`

Here follows the course of action for each of these columns in code.

Since the exploratory data analysis was done using a panadas data frame we will now convert it into a pySpark data fram called `df`.

In [4]:
df = sqlCtx.createDataFrame(data_pd)

Printing the schema. Seems to be ok for most part. The `created` should be changed from string to time object.

In [5]:
df.printSchema()

root
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: long (nullable = true)
 |-- building_id: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- display_address: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- latitude: double (nullable = true)
 |-- listing_id: long (nullable = true)
 |-- longitude: double (nullable = true)
 |-- manager_id: string (nullable = true)
 |-- photos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- price: long (nullable = true)
 |-- street_address: string (nullable = true)
 |-- interest_level: string (nullable = true)



In [5]:
df = df.withColumn("created", F.to_timestamp("created"))
df.cache()

DataFrame[bathrooms: double, bedrooms: bigint, building_id: string, created: timestamp, description: string, display_address: string, features: array<string>, latitude: double, listing_id: bigint, longitude: double, manager_id: string, photos: array<string>, price: bigint, street_address: string, interest_level: string]

## 3.1 The column `features`

The goal here is to divide the different individual features into different clusters. And make 1 new column per cluster in the data set and then fit a regression model on the clusters. How ever the implementation when running the model was a little different

We have to handle the rentals with missing featurses by adding `missing feature` as a feature. Here are the 2 first listings with missing features.

In [7]:
df.filter(F.size("features")==0)[["listing_id", "features"]].show(2)

+----------+--------+
|listing_id|features|
+----------+--------+
|   7211212|      []|
|   6894514|      []|
+----------+--------+
only showing top 2 rows



But first since the column `features` is a list containing strings, we want to mearge the list of strings to one string. We do this by constructing a `UserDefinedFunction` named `string_assembler` which uses the `','.join()` to join strings in a list and separating them with a **,** . This is what we do: \['dog', 'cat'\] -> 'dog,cat'.

In [6]:
string_assembler = F.UserDefinedFunction(lambda x: ','.join(x), typ.StringType())

The function `string_assembler` is used on the column `features`.

In [7]:
df = df.withColumn("features", string_assembler(df["features"]))

All strings are set to lower case.

In [8]:
df = df.withColumn("features", F.lower(df["features"]))

A display of what we have done. Note that the first row is missing any `features`. This is valuable information as well since a listing ad without information about the rental place could determin the popularity of the place.


In [11]:
df[["features"]].show(5)

+--------------------+
|            features|
+--------------------+
|                    |
|doorman,elevator,...|
|laundry in buildi...|
|hardwood floors,n...|
|             pre-war|
+--------------------+
only showing top 5 rows



We impute all missing `features` with the string "missing feature" and show the results.

In [9]:
df = df.withColumn("features", 
                    F.when(df["features"] == '', 'missing features')
                    .otherwise(df["features"]))
df[["features"]].show(5)

+--------------------+
|            features|
+--------------------+
|    missing features|
|doorman,elevator,...|
|laundry in buildi...|
|hardwood floors,n...|
|             pre-war|
+--------------------+
only showing top 5 rows



We want to split the `features` string on **","** and on __"*"__ since some listings had all features in one long string with * separating the different features. 

In [10]:
feat_df = df.withColumn("features_list", F.split(df["features"], ',| \* '))
feat_df[["features_list"]].show(5)

+--------------------+
|       features_list|
+--------------------+
|  [missing features]|
|[doorman, elevato...|
|[laundry in build...|
|[hardwood floors,...|
|           [pre-war]|
+--------------------+
only showing top 5 rows



Now we want to create one feature be uniqe feature in the `features`. The first step is to 'explode' the features so that each single feature gets it's own row.

![Explode%20column.jpg](attachment:Explode%20column.jpg)

In [11]:
feat_df_ex = feat_df.withColumn("ex_features_list", F.explode(feat_df["features_list"]))

We create a data frame called `clustering_df` which contains all different featurese.

In [12]:
clustering_df = feat_df_ex[["ex_features_list"]]
clustering_df.cache()
clustering_df.show(5)

+----------------+
|ex_features_list|
+----------------+
|missing features|
|         doorman|
|        elevator|
|  fitness center|
|    cats allowed|
+----------------+
only showing top 5 rows



Change the name of the column from "ex_features_list" to "text".

In [13]:
clustering_df = clustering_df.withColumnRenamed("ex_features_list", "text")
clustering_df.columns

['text']

Here we create a new data frame `pipelined_df`. In the pipeline we use `Tokenizer`, to divide features with several words into as few meaningful components as possible, `StopWordsRemover` to remove unnecessary words, `HashingTF` to convert the strings into number, and finally, `IDF` (inverse document frequency) which is a measurement of the importance of a word in a document.

In [14]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover
from pyspark.ml.pipeline import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="stopWordsRemovedTokens")
hashingTF = HashingTF(inputCol="stopWordsRemovedTokens", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])
pipelined_df = pipeline.fit(clustering_df).transform(clustering_df)

The pipelined data frame can be seen here:

In [18]:
pipelined_df.show(5)

+----------------+-------------------+----------------------+--------------------+--------------------+
|            text|             tokens|stopWordsRemovedTokens|         rawFeatures|            features|
+----------------+-------------------+----------------------+--------------------+--------------------+
|missing features|[missing, features]|   [missing, features]|(2000,[1525,1755]...|(2000,[1525,1755]...|
|         doorman|          [doorman]|             [doorman]|  (2000,[825],[1.0])|(2000,[825],[2.55...|
|        elevator|         [elevator]|            [elevator]|   (2000,[31],[1.0])|(2000,[31],[2.335...|
|  fitness center|  [fitness, center]|     [fitness, center]|(2000,[1173,1966]...|(2000,[1173,1966]...|
|    cats allowed|    [cats, allowed]|       [cats, allowed]|(2000,[1350,1966]...|(2000,[1350,1966]...|
+----------------+-------------------+----------------------+--------------------+--------------------+
only showing top 5 rows



We are not ready to cluseter the data frame `pipelined_df` which by default uses K-means on the column `features`. `BisectingKMeans` is used for clustering. It was choosen since the normal K-means clustered allmost all observations in the same cluster while `BisectingKMeans` had the observations more eavenly spread out over different clusters.

To find the best number of clusters `k` the elbow method is used. __These lines of code takes time to run and could be skipped__, k = 18 using the Elbow method.

In [19]:
# Sum_of_squared_distances = []
# K = [2,10,12,14,16,17,18,19,20,22,30,40]

# for i in K:
#     km = BisectingKMeans(k = i)
#     model = km.fit(pipelined_df)
#     Sum_of_squared_distances.append(model.computeCost(pipelined_df))
#     print(i)
 
# plt.plot(K, Sum_of_squared_distances, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Sum_of_squared_distances')
# plt.title('Elbow Method For Optimal k')
# plt.show()

From the plot and the elbow method for optimal `k` indikates k = 18.

In [15]:
num_k = 18

We train the model using the `k` choosen by the elbow method.

In [16]:
km = BisectingKMeans(k = num_k)
model = km.fit(pipelined_df)

The results are printed.

In [17]:
results = model.transform(pipelined_df)
results.cache()
results.show(5)

+----------------+-------------------+----------------------+--------------------+--------------------+----------+
|            text|             tokens|stopWordsRemovedTokens|         rawFeatures|            features|prediction|
+----------------+-------------------+----------------------+--------------------+--------------------+----------+
|missing features|[missing, features]|   [missing, features]|(2000,[1525,1755]...|(2000,[1525,1755]...|        16|
|         doorman|          [doorman]|             [doorman]|  (2000,[825],[1.0])|(2000,[825],[2.55...|         0|
|        elevator|         [elevator]|            [elevator]|   (2000,[31],[1.0])|(2000,[31],[2.335...|         3|
|  fitness center|  [fitness, center]|     [fitness, center]|(2000,[1173,1966]...|(2000,[1173,1966]...|         2|
|    cats allowed|    [cats, allowed]|       [cats, allowed]|(2000,[1350,1966]...|(2000,[1350,1966]...|        13|
+----------------+-------------------+----------------------+-------------------

The column default output column `precidtion` is changed to `clusters`.

In [18]:
results = results.withColumnRenamed("prediction", "clusters")

In [19]:
join_df = results.drop(*["tokens", "stopWordsRemovedTokens", "rawFeatures", "features"])
join_df.show(10)

+--------------------+--------+
|                text|clusters|
+--------------------+--------+
|    missing features|      16|
|             doorman|       0|
|            elevator|       3|
|      fitness center|       2|
|        cats allowed|      13|
|        dogs allowed|      10|
| laundry in building|      14|
|          dishwasher|       0|
|     hardwood floors|       5|
|pets allowed case...|      13|
+--------------------+--------+
only showing top 10 rows



Some examples of cluseters are displayed bellow where we can see what features get clustered together.

In [25]:
print("Doorman cluster:")
join_df.filter(join_df["clusters"] == 0).distinct().show(5)
print("Loft cluster:")
join_df.filter(join_df["clusters"] == 1).distinct().show(5)
print("Washer cluster:")
join_df.filter(join_df["clusters"] == 10).distinct().show(5)

Doorman cluster:
+--------------------+--------+
|                text|clusters|
+--------------------+--------+
|• on-site lifesty...|       0|
|dogs under 20 lbs...|       0|
|  short term allowed|       0|
|  pre-war small dogs|       0|
|       small dogs ok|       0|
+--------------------+--------+
only showing top 5 rows

Loft cluster:
+--------+--------+
|    text|clusters|
+--------+--------+
|elevator|       1|
+--------+--------+

Washer cluster:
+--------------------+--------+
|                text|clusters|
+--------------------+--------+
|        24hr doorman|      10|
|24hr white-gloved...|      10|
|   full-time doorman|      10|
|   part-time doorman|      10|
|        24/7 doorman|      10|
+--------------------+--------+
only showing top 5 rows



Now we want to join the two data sets together and create a column to add on.

In [20]:
join_df = join_df.withColumn("join_col", F.monotonically_increasing_id())
feat_df_ex = feat_df_ex.withColumn("join_col", F.monotonically_increasing_id())

The 2 data frames are shown bellow.

In [27]:
join_df.show(3)
feat_df_ex[["ex_features_list", "join_col"]].show(3)

+----------------+--------+--------+
|            text|clusters|join_col|
+----------------+--------+--------+
|missing features|       9|       0|
|         doorman|      10|       1|
|        elevator|       1|       2|
+----------------+--------+--------+
only showing top 3 rows

+----------------+--------+
|ex_features_list|join_col|
+----------------+--------+
|missing features|       0|
|         doorman|       1|
|        elevator|       2|
+----------------+--------+
only showing top 3 rows



We join the 2 data frames based on the created column, which after the join is droped. The columns are displayed below.

In [21]:
joined_df = feat_df_ex.join(join_df, feat_df_ex["join_col"] == join_df["join_col"], how = "left")
joined_df = joined_df.drop("join_col")

The columns `text` and `prediction` are shown.

In [22]:
joined_df[["listing_id", "text", "clusters"]].show(5)

+----------+---------------+--------+
|listing_id|           text|clusters|
+----------+---------------+--------+
|   6867392|laundry in unit|      15|
|   6867392|  outdoor space|       6|
|   6848778|laundry in unit|      15|
|   6862351|   cats allowed|      13|
|   6898347|on-site laundry|      14|
+----------+---------------+--------+
only showing top 5 rows



We then use the function `collect_set` to assemble the clusters assosiated with each `listing_id`.

In [23]:
collect_df = joined_df.groupBy('listing_id').agg(collect_set('clusters').alias('clusters_list'))
collect_df.show(5)

+----------+--------------------+
|listing_id|       clusters_list|
+----------+--------------------+
|   6813946|[0, 13, 5, 3, 10,...|
|   6816559|[0, 5, 2, 17, 3, ...|
|   6817923|        [12, 13, 10]|
|   6819131|[0, 15, 5, 2, 17,...|
|   6820763| [0, 5, 2, 3, 14, 8]|
+----------+--------------------+
only showing top 5 rows



Finaly we add the new column to the data set.

In [44]:
new_df = df.join(collect_df, on = "listing_id", how = "left")
new_df.columns

['listing_id',
 'bathrooms',
 'bedrooms',
 'building_id',
 'created',
 'description',
 'display_address',
 'features',
 'latitude',
 'longitude',
 'manager_id',
 'photos',
 'price',
 'street_address',
 'interest_level',
 'clusters_list']

We conclude that the column `clusters_list` has been added to the data frame and store the final data frame as `amenities_df` for later use.

In [45]:
amenities_df = new_df # rename for compatibility

In [46]:
amenities_df.show(2)

+----------+---------+--------+--------------------+-------------------+--------------------+---------------+--------------------+--------+---------+--------------------+--------------------+-----+---------------+--------------+--------------------+
|listing_id|bathrooms|bedrooms|         building_id|            created|         description|display_address|            features|latitude|longitude|          manager_id|              photos|price| street_address|interest_level|       clusters_list|
+----------+---------+--------+--------------------+-------------------+--------------------+---------------+--------------------+--------+---------+--------------------+--------------------+-----+---------------+--------------+--------------------+
|   6813946|      1.0|       0|e84a788c193121f20...|2016-04-02 02:50:04|Renovated studio ...|    W 44 Street|roof deck,elevato...| 40.7609| -73.9941|624c1fbd75e5f99e6...|[https://photos.r...| 1995|461 W 44 Street|           low|[0, 13, 5, 3, 10,...|


## 3.2 The column `manager_id`


The managers are not equally skilled in making an add for the rental unit that would cause a high interest level. Therefore we can create a new feature that captures the skill of the manager. 


We create a new column `count` where the number of rentals for each manager is.

In [ ]:
manager_df = df
join_df = manager_df[["manager_id"]].groupBy("manager_id").count()
manager_df = manager_df.join(join_df, on = "manager_id")

In [ ]:
df[["manager_id"]].groupBy("manager_id").count().sort("count", ascending = False).show(4)

The `interest_level` are changed into low = 0, medium = 1, high = 2. 

In [ ]:
manager_df = manager_df.withColumn("interest_level", F.when(manager_df["interest_level"] == 'low', 0)
                                         .when(manager_df["interest_level"] == 'medium', 1)
                                         .otherwise(2))
manager_df.cache()
manager_df[["manager_id", "count", "interest_level"]].show(4)

We create a data frame `manager_skill` where for each maneger their skill `manager_skill` was calculated by summing up all of their rentals and give 2 points for interest level "high", 1 point for interest level "medium" and 0 points for interest level "low" and then taking the mean.

In [ ]:
manager_skill = manager_df.groupBy("manager_id").agg({"interest_level": "mean"})
manager_skill.show(5)

The name of the column is changed from `avg(interest_level)` to `manager_skill`.

In [ ]:
manager_skill = manager_skill.withColumnRenamed("avg(interest_level)", "manager_skill")

We join the dataframes so there is a column `manager_skill` for the original data.

In [ ]:
manager_df = manager_df.join(manager_skill, on = "manager_id", how = "left")
manager_df[["interest_level", "manager_id", "manager_skill"]].show(3)

Lastly we bucket the manager skills into different buckets.   
__Note:__ Here the buckets get cast into 3 bucets choosen by a person. The ranges of the buckets should be choosen by K-means or Gaussian Mixture Models. The bucketing was a last minute change which improved the results over not using bucketing anyways. That is the reason the choosing of buckets was not implemented using K-means or Gaussian Mixture Models.

In [ ]:
splits = [0,0.66,1.33,float("Inf")]
buck = Bucketizer(splits = splits, inputCol = "manager_skill", outputCol = "manager_bucket")
manager_skill_bucket = buck.transform(manager_skill)
manager_skill_bucket.show(5)

Lastly we add the new column `manager_bucket` and `manager_skill` to the original data frame.

In [ ]:
new_df = df.join(manager_skill_bucket, on = "manager_id")
new_df.columns

We see that the 2 columns have been added to the data frame. And save it as `df_amenities` for later use.

In [57]:
df_amenities = new_df

## 3.3 Time created

For both model tesing the prediction, the **created** is converted to the unix time, so that becomes a real number suitable for various of models. 

In [ ]:
! cat projectModelRunner.py | grep "mktime"

## 3.4 latitude and longitude

In [ ]:
! head -49 Miscellaneous/ModelPipConfig.py | tail -15

Because we assume that houses' 2D locations with different interest level are distributed in NYC by different Gaussian mixtures. Points are clustered and one hot encoded and then fitted into the model

## Extreme Value Imputation

I believe that observations with a modified z-score (based on the median absolute deviation) greater than 5.5 are extreme outliers and are entered mistakenly. I wrote the following code for cleaning those outliers. 

In [ ]:
! cat FeaturesMakers/OutlierSmoother.py

Basically, the above code will set each of the extreme outliers' values to the average of the remaining data. 

## 3.5 Description Column

We conducted three different methods of text analysis and used the best one in our final model:
1. Count Vector
2. TF-IDF
3. Word2Vec

### 3.5.1 Count Vector

We first use the simple count vector method.

In [31]:
# Make pyspark df using the updated dataframe
train_data_df = sqlCtx.createDataFrame(data_pd)

In [32]:
# Change the response to integer values
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def modify_values(r):
    if r == "high":
        return 2
    else:
        if r == "medium":
            return 1
        else:
            return 0
ol_val = udf(modify_values, IntegerType())
new_df2 = train_data_df.withColumn("label",ol_val(train_data_df.interest_level))

Here we create a pipeline that tokenizes the words, removes stopwords, counts the words, and generates an logistic regression model.

In [33]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W") # I don't know what W is...
# stop words
add_stopwords = ["a","the","it","of","the","is","and","A","this","in","for"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# count vectors
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
# logistic regression
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0, family = "multinomial")
    
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, mlr])

We now split the data into the training and testing sets; however, we will do this twice to ensure that we are not making model-builing decisions (i.e. selecting the best text-analysis model) from the testing set.

In [35]:
# First select only the description and output
(train_set, test_set) = new_df2.select("description","label").randomSplit([0.8, 0.2], seed = 1337) # first split
(train2, test2) = train_set.randomSplit([0.8, 0.2], seed = 1000) # second split on training data only

In [ ]:
train2.show(5)

We now make a simple cross-validation grid to determine the best model. Specifically, we will see if changing the regularization parameter and vocab size can improve predictions.

In [37]:
# This takes a couple of minutes to run
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(countVectors.vocabSize, [1000, 5000, 10000]) \
    .addGrid(mlr.regParam, [0.1, 0.3]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel = crossval.fit(train2)

In [ ]:
prediction = cvModel.transform(test2)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction)))

In [ ]:
We created a function to calculate the log-loss:

In [ ]:
from numpy import log

def log_loss(prediction, labels = "label", probability = "probability"):    
    labs_and_preds = prediction[labels, probability]
    
    return - labs_and_preds\
                .rdd\
                .map(lambda x: log(x[1][x[0]]))\
                .reduce(lambda x,y: x + y) / labs_and_preds.count()

In [ ]:
log_loss(prediction)

In [ ]:
We can also examine model details to see which parameters generated the best model:

In [ ]:
# Details on the model:
print ("Best regularization parameter value:" + str(cvModel.bestModel.stages[-1]._java_obj.parent().getRegParam()))
print ("Best number of words:" + str(cvModel.bestModel.stages[2]._java_obj.parent().getVocabSize()))

### 3.5.2 TF-IDF

The next method we considered was the TF-IDF method, which accounts for word frequencies more delicately than the count vectorizer. First we make a new pipeline:

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer

tokenizer = Tokenizer(inputCol="description", outputCol="words")
hashtf = HashingTF(numFeatures=10000, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0, family = "multinomial")

pipeline = Pipeline(stages=[tokenizer, hashtf, idf, mlr])

We also update the parameter grid to see if varying the number of features and minimum document frequency improves predictions.

In [ ]:
# This takes about 10-15 minutes to run
paramGrid = ParamGridBuilder() \
    .addGrid(hashtf.numFeatures, [1000, 5000, 10000]) \
    .addGrid(idf.minDocFreq, [2, 5])\
    .addGrid(mlr.regParam, [0.1, 0.3]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel2 = crossval.fit(train2)

In [ ]:
prediction2 = cvModel2.transform(test2)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction2)))

In [ ]:
log_loss(prediction2)

In [ ]:
#print("Best number of features:" + str(cvModel2.bestModel.stages[1]._java_obj.parent().getNumFeatures()))
print("Best min doc frequency:" + str(cvModel2.bestModel.stages[2]._java_obj.parent().getMinDocFreq()))
print("Best regularization:" + str(cvModel2.bestModel.stages[-1]._java_obj.parent().getRegParam()))

### 3.5.3 Word2Vec

Finally, we consider the Word2Vec method, which accounts for a word's context in making predictions.

In [ ]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=50, minCount=5, inputCol="filtered", outputCol="features")
# It takes way too long if we use a vector that's much larger than this

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, word2Vec, mlr])

This method is computationally-expensive, so we are limited in the parameter space tested using cross-validation. Luckily, we know that the average description is about 80-100 words, so vectors do not need to be huge.

In [ ]:
# This takes several minutes to run
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(word2Vec.vectorSize, [10,50]) \
    .addGrid(mlr.regParam, [0.1, 0.3]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel3 = crossval.fit(train2)

In [ ]:
prediction3 = cvModel3.transform(test2)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction3)))

In [ ]:
log_loss(prediction3)

In [ ]:
print("Best vector size:" + str(cvModel3.bestModel.stages[2]._java_obj.parent().getVectorSize()))
print("Best regularization:" + str(cvModel3.bestModel.stages[-1]._java_obj.parent().getRegParam()))

# 4. Model Selection, Parameter Tunning, and Cross Validation

We compared several different models from three different machine-learning families: Multinomial logistic regression, random forests, and decision trees.
    For each of these model types, we compared 1) a simple baseline model with bedrooms, bathrooms, price, and location clustering 2) Amenities 3) Description and 4) All models together.

## 4.1 Baseline model

### 4.1.1 Baseline logistic

In [ ]:
! python3 projectModelRunner.py finalClassifier:LogisticRegression doTest:true

### 4.1.2 Baseline random forest

In [ ]:
! python3 projectModelRunner.py finalClassifier:RandomForest doTest:true

### 4.1.3 Baseline decision tree

In [ ]:
! python3 projectModelRunner.py finalClassifier:DecisionTree doTest:true

The model selected in this case is Random Forest. Because there are some "cutting points" in the plots, a tree model might help. More trees will increase the robustness of the prediction, and here we use 10 trees. 

Since we have clusterd the **latitude** and **longitude** with GMM, the number of clusters are unknown, so we need to tune that also. 

In [ ]:
! head -73 Miscellaneous/ModelPipConfig.py | tail -5

So we tune the number of clusters from 3 to 10

The cross validation is used to tune the parameters. In this case, we are using the logloss which is defined as follows:

$$
log loss = -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^My_{ij}\log(p_{ij}),
$$

The log loss is a metric that gives an insidation of how correct your model is and how certain it is of beeing correct. Below is an implementation of calculation of the _logloss_ called `log_loss`. The function takes in the data frame that is created when we call the transform on a created model for the data set we want, i.e. model.transform(test)

In [ ]:
from numpy import log

def log_loss(results_transformed, lables = "label", probability = "probability"):    
    labs_and_preds = results_transformed[lables, probability]
    
    return - labs_and_preds\
                .rdd\
                .map(lambda x: log(x[1][x[0]]))\
                .reduce(lambda x,y: x + y) / labs_and_preds.count()

In [ ]:
! cat ModelEvaluators/MultiClassLogLossEvaluator.py

This metric is better when it is smaller. And the above evaluator is passed in the cross validator

In [ ]:
! head -58 Miscellaneous/ModelPipConfig.py | tail -1

In [ ]:
! head -133 projectModelRunner.py | tail -5

In [ ]:
! tail project.log

Here is the confusion matrix based on the default threshold. The last line of this file gives a variable importance measure for each variable used in the random forest stage of the pipeline

In [ ]:
! head -56 Miscellaneous/ModelPipConfig.py | tail -8

Here is the order of the variables that we fitted into the random forest classifier. And as you can see, price is the most important variable follows by number of bathrooms and number of bedrooms. The optimal number of the GMM mixtures after the tunning is 2. Time created and GMM clusters are important, even more important than the number of bathrooms. 

# The business goal

The business goal for this project is to predict the interest level of each listing of houses in NYC given their listing information. 
And we have generated estimated probabilities of each listing for different use cases. 

In [ ]:
probabilities = list(map(eval,pd.read_csv("predictionOutput.csv")["probability"].tolist()))
np.array(probabilities)

In [ ]:
labels = pd.read_csv("predictionOutput.csv")["label"].tolist()

In [ ]:
def getPredictionLabel(probabilities, weight):
    #make sure the summation of thresh is 1
    #make sure the indicies correspond with labels
    output = []
    for currProbVec in probabilities:
        output.append(np.argmax( np.array(currProbVec)*np.array(weight) ))
    return output

We want to plot a nice looking confusion matrix. This is done by the function `plot_confusion_matrix` implemented bellow. The function is taken from https://scikit-learn.org and the link to the function is here: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html 

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
class_names = ["low", "medium", "high"]

In [ ]:
cm = confusion_matrix(labels,getPredictionLabel(probabilities, [1,1,1]))
plot_confusion_matrix(cm, classes=class_names, normalize = False,
                      title='Confusion matrix, without normalization')

The above confusion matrix is generated by using the default threshold. 
We have obtained 77/(77+45+31) percition and 77/(77+11+687) recall. Which is quite good actually. 

To increase the recall, we can add more weight to the first label

In [ ]:
cm = confusion_matrix(labels,getPredictionLabel(probabilities, [4,1,1]))
plot_confusion_matrix(cm, classes=class_names, normalize = False,
                      title='Confusion matrix, without normalization')

Now the recall increased to about 50 percent, and precision is

In [ ]:
252/(252+295+346)

Which is still not bad... And actually 28 percent precision is far above the threshold for generating leads for a shortening a sales cycle. 

If we want to increase the precision, we can do:

In [ ]:
cm = confusion_matrix(labels,getPredictionLabel(probabilities, [0.5,1,1]))
plot_confusion_matrix(cm, classes=class_names, normalize = False,
                      title='Confusion matrix, without normalization')

## 4.2 Amenities only

### 4.2.1 Amenities logistic

We rejoined the amenity cluster assignments to the listing labels, creating a column for each cluster assignment. We then combined these assigments into a feature vector and reassigned integer values to the label, split into training and testing sets, and create a new pipeline for crossvalidation.

In [71]:
results = results.withColumnRenamed("prediction", "clusters")

In [72]:
results.show(2)

+----------------+-------------------+----------------------+--------------------+--------------------+--------+
|            text|             tokens|stopWordsRemovedTokens|         rawFeatures|            features|clusters|
+----------------+-------------------+----------------------+--------------------+--------------------+--------+
|missing features|[missing, features]|   [missing, features]|(2000,[1525,1755]...|(2000,[1525,1755]...|      16|
|         doorman|          [doorman]|             [doorman]|  (2000,[825],[1.0])|(2000,[825],[2.55...|       0|
+----------------+-------------------+----------------------+--------------------+--------------------+--------+
only showing top 2 rows



In [73]:
join_df = results.drop(*["tokens", "stopWordsRemovedTokens", "rawFeatures", "features"])
join_df.show(2)

+----------------+--------+
|            text|clusters|
+----------------+--------+
|missing features|      16|
|         doorman|       0|
+----------------+--------+
only showing top 2 rows



In [74]:
join_df = join_df.withColumn("join_col", F.monotonically_increasing_id())
feat_df_ex = feat_df_ex.withColumn("join_col", F.monotonically_increasing_id())

In [75]:
joined_df = feat_df_ex.join(join_df, feat_df_ex["join_col"] == join_df["join_col"], how = "left")
joined_df = joined_df.drop("join_col")

In [76]:
joined_df = joined_df.withColumn("constant_val", F.lit(1))

In [77]:
df_piv = joined_df\
                   .groupBy("listing_id")\
                   .pivot("clusters")\
                   .agg(F.coalesce(F.first("constant_val")))
df_piv.cache()
df_piv.count()

49352

In [78]:
joined_df = train_data_df.join(df_piv, on = "listing_id", how = "left")
joined_df.cache()
print(joined_df.count())
print(len(joined_df.columns))

49352
33


In [79]:
cluster_col = df_piv.columns
cluster_col.remove("listing_id")

In [80]:
joined_df = joined_df.fillna(0, subset = cluster_col)

In [81]:
cols_to_drop = train_data_df.columns
cols_to_drop.remove("interest_level")
cluseters_df = joined_df.drop(*cols_to_drop)

In [82]:
imput_data = cluseters_df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
df_dense = sqlCtx.createDataFrame(imput_data, ['label','features'])

In [83]:
df_dense.show(2)

+------+--------------------+
| label|            features|
+------+--------------------+
|medium|[0.0,0.0,0.0,0.0,...|
|   low|[1.0,0.0,1.0,1.0,...|
+------+--------------------+
only showing top 2 rows



In [84]:
df_dense = df_dense.withColumn("label", F.when(df_dense["label"] == 'low', 0)
                                         .when(df_dense["label"] == 'medium', 1)
                                         .otherwise(2))
df_dense.cache()

DataFrame[label: int, features: vector]

In [85]:
df_dense.show(1)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 1 row



In [89]:
# Split data
(train, test) = df_dense.select("features","label").randomSplit(seed = 1337, weights = [0.8, 0.2])

In [87]:
# new pipeline
mlr = LogisticRegression(maxIter=10, regParam = 0.1, family="multinomial")
pipeline2 = Pipeline(stages=[mlr])

In [90]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Create a new parameter grid for model optimization
paramGrid = ParamGridBuilder() \
    .addGrid(mlr.regParam, [0.01, 0.1, 0.3])\
    .build()

crossval = CrossValidator(estimator=pipeline2,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel4 = crossval.fit(train)

In [ ]:
prediction4 = cvModel4.transform(test)

evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction4)))

In [ ]:
log_loss(prediction4)

In [ ]:
cvModel4.bestModel.stages[-1]._java_obj.parent().getRegParam()

### 4.2.2 Amenities random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(maxDepth=5, numTrees=50)
pipeline2 = Pipeline(stages=[rf])

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [2, 5]) \
    .addGrid(rf.numTrees, [50, 100])\
    .build()

crossval = CrossValidator(estimator=pipeline2,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel5 = crossval.fit(train)

In [ ]:
prediction5 = cvModel5.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction6)))

In [ ]:
log_loss(prediction5)

In [ ]:
cvModel5.bestModel.stages[-1]._java_obj.parent().getMaxDepth()
cvModel5.bestModel.stages[-1]._java_obj.parent().getNumTrees()

### 4.2.3 Amenities decision tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(maxDepth=5)
pipeline = Pipeline(stages=[dt])

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [2, 5]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel6 = crossval.fit(train)

In [ ]:
prediction6 = cvModel6.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction8)))

In [ ]:
log_loss(prediction6)

In [ ]:
cvModel6.bestModel.stages[-1]._java_obj.parent().getMaxDepth()

## 4.3 Description

### 4.3.1 Description Logistic
We have already run this once to determine that the count-vector method is the best. Here we will refit the model using the full training set and evaluate on the testing set.

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W") # I don't know what W is...
# stop words
add_stopwords = ["a","the","it","of","the","is","and","A","this","in","for"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# count vectors
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
# logistic regression
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0, family = "multinomial")
    
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors,mlr])

We now use the full training set to train the model using the best parameters from cross-validation

In [ ]:
# This takes a couple of minutes to run
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(countVectors.vocabSize, [5000, 10000]) \
    .addGrid(mlr.regParam, [0.1, 0.3]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel7 = crossval.fit(train2)

In [ ]:
prediction7 = cvModel7.transform(test2)

In [ ]:
# Details on the model:
print ("Best regularization parameter value:" + str(cvModel.bestModel.stages[-1]._java_obj.parent().getRegParam()))
print ("Best number of words:" + str(cvModel.bestModel.stages[2]._java_obj.parent().getVocabSize()))

In [ ]:
log_loss(prediction7)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction7)))

### 4.3.2 Description Random Forest

Now we would like to see how the Count Vector method works using a Random Forest classifier. We will perform cross-validation for this step.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier


In [ ]:
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W") # I don't know what W is...

# stop words
add_stopwords = ["a","the","it","of","the","is","and","A","this","in","for", "with"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=5000, minDF=5)

rf = RandomForestClassifier(maxDepth=5, numTrees=50)

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, rf])


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


paramGrid = ParamGridBuilder() \
    .addGrid(countVectors.vocabSize, [1000, 5000]) \
    .addGrid(rf.maxDepth, [2, 5]) \
    .addGrid(rf.numTrees, [50, 100])\
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel8 = crossval.fit(train_set)

In [ ]:
prediction8 = cvModel8.transform(test_set)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction8)))

In [ ]:
log_loss(prediction8) # note, this is different from the first time I ran this

In [ ]:
cvModel8.bestModel.stages[-1]._java_obj.parent().getMaxDepth()
cvModel8.bestModel.stages[-1]._java_obj.parent().getNumTrees()
cvModel8.bestModel.stages[2]._java_obj.parent().getVocabSize()

### 4.3.3 Description Decision Tree

In [ ]:
# Decision Trees
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(maxDepth=5)
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, dt])

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


paramGrid = ParamGridBuilder() \
    .addGrid(countVectors.vocabSize, [1000, 5000]) \
    .addGrid(dt.maxDepth, [2, 5]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=5)

cvModel9 = crossval.fit(train_set)

In [ ]:
prediction9 = cvModel9.transform(test_set)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(prediction9)))

In [ ]:
log_loss(prediction9) # 

In [ ]:
cvModel9.bestModel.stages[-1]._java_obj.parent().getMaxDepth()
cvModel9.bestModel.stages[2]._java_obj.parent().getVocabSize()

## 4.4 Mega-Model

### Logistic Regression

In [ ]:
!python3 projectModelRunnerMega.py finalClassifier:LogisticRegression doTest:true 

### Decision Tree

In [ ]:
!python3 projectModelRunnerMega.py finalClassifier:DecisionTree doTest:true 

### Random Forest

In [ ]:
!python3 projectModelRunnerMega.py finalClassifier:RandomForest doTest:true 

# 5. Evaluation

We created a confusion matrix to visualize performance of the best models from each model type.

In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

First, the amenities column: (logistic)

In [ ]:
test_confusion_matrix_pd = prediction4.select("label", "prediction").toPandas()

from sklearn.metrics import confusion_matrix


cnf_matrix = confusion_matrix(test_confusion_matrix_pd["label"], test_confusion_matrix_pd["prediction"])

plt.figure()
class_names = ["low", "medium", "high"]
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize = False,
                      title='Confusion matrix, without normalization')

In [ ]:
The description column: (logistic)

In [ ]:
test_confusion_matrix_pd = prediction7.select("label", "prediction").toPandas()

from sklearn.metrics import confusion_matrix


cnf_matrix = confusion_matrix(test_confusion_matrix_pd["label"], test_confusion_matrix_pd["prediction"])

plt.figure()
class_names = ["low", "medium", "high"]
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize = False,
                      title='Confusion matrix, without normalization')

# 6. Additional work

## 6.1 Time
We have fitted the time variable in the random forest as the unix time. In the random forest setting, that time will be cutted by the mean in between each two observations at each step chosen by the lowest Gini value. However, by this method the time variable is not important in this model. We may fit time as categories such as Months of a year, and that may improve the model performance.  

## 6.2 Extract 'Hot' Neighborhoods

We expected that neighborhood designation might be an important predictor in the model. However, the dataset provided did not include this information. Therefore, we mined the description column for words that indicated which NYC neighborhood the listing might be in. We went to rentHop's website and searched the text for the neighborhoods used online.

In [ ]:
# These are all the neighborhoods listed on the renthop website that users can use to filter
def hot_hoods(df):
    train5 = df.withColumn('Lenox_Hill', F.expr("IF(INSTR(description, 'Lenox Hill') > 0, 1, 0)"))\
    .withColumn('Roosevelt_Island', F.expr("IF(INSTR(description, 'Roosevelt Island') > 0, 1, 0)"))\
    .withColumn('CP', F.expr("IF(INSTR(description, 'Central Park') >0,1,0)"))\
    .withColumn('Turtle_Bay', F.expr("IF(INSTR(description, 'Turtle Bay') > 0, 1, 0)"))\
    .withColumn('Upper_East', F.expr("IF(INSTR(description, 'Upper East')|INSTR(description, 'upper East') >0,1,0)"))\
    .withColumn('Upper_West', F.expr("IF(INSTR(description, 'Upper West')|INSTR(description, 'upper East') >0,1,0)"))\
    .withColumn('Brooklyn', F.expr("IF(INSTR(description, 'Brooklyn') >0,1,0)"))\
    .withColumn('HK', F.expr("IF(INSTR(description, 'Hell') >0,1,0)"))\
    .withColumn('Midtown', F.expr("IF(INSTR(description, 'Midtown') >0,1,0)"))\
    .withColumn('CH', F.expr("IF(INSTR(description, 'Carnegie Hill') >0,1,0)"))\
    .withColumn('GD', F.expr("IF(INSTR(description, 'Garment') >0,1,0)"))\
    .withColumn('MHill', F.expr("IF(INSTR(description, 'Murray') >0,1,0)"))\
    .withColumn('Yorkville', F.expr("IF(INSTR(description, 'Yorkville') >0,1,0)"))\
    .withColumn('TD', F.expr("IF(INSTR(description, 'Theater District') >0,1,0)"))\
    .withColumn('SP', F.expr("IF(INSTR(description, 'Sutton Place') >0,1,0)"))\
    .withColumn('Manhattan', F.expr("IF(INSTR(description, 'Manhattan') >0,1,0)"))\
    .withColumn('TC', F.expr("IF(INSTR(description, 'Tudor') >0,1,0)"))\
    .withColumn('Koreatown', F.expr("IF(INSTR(description, 'Koreatown') >0,1,0)"))\
    .withColumn('LS', F.expr("IF(INSTR(description, 'Little Senegal') >0,1,0)"))\
    .withColumn('Lincoln_Sq', F.expr("IF(INSTR(description, 'Lincoln Square') >0,1,0)"))
    
    return train5

In [ ]:
dataset_3 = hot_hoods(train_data_df)

In [ ]:
dataset_3.printSchema

We can even build a quick model to see how it does:

In [ ]:
# Make input/output vectors
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import DenseVector


dataset_3B = dataset_3.select("interest_level", "price", "Lenox_Hill","Roosevelt_Island","CP","Turtle_Bay","Upper_East","Upper_West","Brooklyn","HK","Midtown","CH","GD","MHill","Yorkville","TD","SP","Manhattan","TC","Koreatown","LS","Lincoln_Sq")
input_data = dataset_3B.rdd.map(lambda x: (x[0], DenseVector(x[2:])))
df2 = sqlCtx.createDataFrame(input_data, ["label", "features"])
df2 = withColumn("label", )
df2.show(2)

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def modify_values(r):
    if r == "high":
        return 2
    else:
        if r == "medium":
            return 1
        else:
            return 0
ol_val = udf(modify_values, IntegerType())
new_df2 = df2.withColumn("label",ol_val(df2.label))

In [ ]:
(train_set2, test_set2) = new_df2.randomSplit([0.8, 0.2], seed = 1337)

In [ ]:
# new pipeline
mlr = LogisticRegression(maxIter=10, regParam = 0.1, family="multinomial")
pipeline2 = Pipeline(stages=[mlr])

In [ ]:
mod = pipeline2.fit(train_set2)

In [ ]:
preds = mod.transform(test_set2)

In [ ]:
evaluator = MulticlassClassificationEvaluator()
print("testing set F1:" + str(evaluator.evaluate(preds)))

We can see that extracting the neighborhoods does not do too much to improve predictions! We can play around with using different regularization levels and thresholds, but this is unilikely to solve all of our prediction problems!

## 6.3 Symbols and text attributes

While word analysis might be very helpful, there might be other elements that influence a person's interest in a listing aside from actual words. Use of '******' might be eye-catching, and too many exclamations might be viewed as unprofessional. Additionally, excessive all caps words might also be seen as more likely to be a scam.

## 6.4 Count Photos

People love pictures, so it's probably likely that potential renters won't be too interested in listings with zero pictures. We'd hazard a guess that listings with three or fewer photos might have less interest, but that beyond 7-8 it probably doesn't make too much difference. This nonlinear relationship might warrant bucketizing, but for now, we will simply use photo counts as a predictor.

In [ ]:
def photo_taker(df):
    train_data_df_p = df.select("interest_level", "photos")
    photo_rdd = train_data_df_p.rdd
    photo_rdd = photo_rdd.map(lambda x: (x[0], len(x[1])))
    photo_df = sqlCtx.createDataFrame(photo_rdd, ["label", "features"])
    return photo_df["features"]

In [ ]:
# Need to extract photo_df from above to run this part to make the histogram
photo_pd = photo_df.toPandas()
photo_pd.hist("features","label")